<a href="https://colab.research.google.com/github/benjuarez8/CS81C-Research-Bolivia/blob/main/cs81c_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
pip install Unidecode

In [2]:
import pandas as pd
import numpy as np
import csv
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from unidecode import unidecode

#Load Data

In [3]:
training_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/training_data.csv")
training_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )
testing_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/testing_data.csv")
testing_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )

#Data Processing

In [4]:
training_data["Fraud_num"] = [ -1 if type == "VS" else 1 if type == "BBS" else 0 for type in training_data["Fraud.type"] ]

In [5]:
training_departments = training_data["Department"].str.get_dummies()
training_data = pd.concat([training_data, training_departments], axis=1)
testing_departments = testing_data["Department"].str.get_dummies()
testing_data = pd.concat([testing_data, testing_departments], axis=1)
testing_data.rename( columns={"Potosí":unidecode("Potosí")}, inplace=True)

In [6]:
train_labels = training_data["Fraud_num"]
train = training_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]
test = testing_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]

In [7]:
X_train = np.array(train)
y_train = np.array(train_labels)
X_test = np.array(test)

#Classification Models

In [8]:
seed = 8
rf_model = RandomForestClassifier()
gb_model = GradientBoostingClassifier()
knn_model = KNeighborsClassifier()
logr_model = LogisticRegression(max_iter = 1000)
xgb_model = XGBClassifier()

# Ensemble (averaging approach)

In [9]:
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
knn_model.fit(X_train, y_train)
logr_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [10]:
rf_pred = rf_model.predict(X_test)
gb_pred = gb_model.predict(X_test)
knn_pred = knn_model.predict(X_test)
logr_pred = logr_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

In [11]:
avg_pred = (rf_pred + gb_pred + knn_pred + logr_pred + xgb_pred) // 5

In [12]:
test["Pred_Fraud_num"] = avg_pred.tolist()
test["Pred_Fraud_type"] = [ "VS" if type == -1 else "BBS" if type == 1 else "Clean" for type in test["Pred_Fraud_num"] ]
test["Department"] = testing_data["Department"]
test["Precinct"] = testing_data["Precinct"]
test["Municipality"] = testing_data["Municipality"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [13]:
test.to_csv("ensemble_avg_results.csv", index=False)

In [14]:
col_names = ["Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
avg_ensemble_results = pd.DataFrame(columns = col_names)
counts = test.Pred_Fraud_type.value_counts()
clean = counts[0] / (counts[0] + counts[1] + counts[2])
tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
bbs = counts[1] / (counts[0] + counts[1] + counts[2])
vs = counts[2] / (counts[0] + counts[1] + counts[2])
avg_turnout = test["Turnout"].mean()
avg_vote = test["MAS.vote"].mean()
avg_ensemble_results.loc[len(avg_ensemble_results)] = [round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]

In [15]:
avg_ensemble_results.to_csv("ensemble_avg_tot_results.csv", index=False)
avg_ensemble_results

,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,89.69,10.31,5.65,4.66,89.8,0.46


In [16]:
col_names = ["Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
avg_ensemble_dep_results = pd.DataFrame(columns = col_names)
for dep in test["Department"].unique():
    vars()[dep] = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    counts = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Department"] == dep]["Turnout"].mean()
    avg_vote = test[test["Department"] == dep]["MAS.vote"].mean()
    avg_ensemble_dep_results.loc[len(avg_ensemble_dep_results)] = [dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
avg_ensemble_dep_results = avg_ensemble_dep_results.sort_values(by = "Clean")
avg_ensemble_dep_results.reset_index(drop=True, inplace=True)

In [17]:
avg_ensemble_dep_results.to_csv("ensemble_avg_dep_results.csv", index=False)
avg_ensemble_dep_results

,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Cochabamba,82.43,17.57,12.87,4.70,90.28,0.57
1,Potosí,86.31,13.69,10.09,3.61,88.87,0.46
2,La Paz,89.10,10.90,6.88,4.02,91.75,0.53
3,Oruro,91.02,8.98,6.18,2.80,91.09,0.47
4,Chuquisaca,91.71,8.29,6.44,1.85,88.41,0.40
5,Santa Cruz,93.74,6.26,3.74,2.52,88.83,0.34
6,Tarija,94.30,5.70,3.45,2.25,87.26,0.39
7,Pando,95.04,4.96,3.05,1.91,84.31,0.43
8,Beni,95.34,4.66,2.98,1.68,86.17,0.34


In [18]:
col_names = ["Municipality", "Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
avg_ensemble_municip_results = pd.DataFrame(columns = col_names)
for municip in test["Municipality"].unique():
  dep = test.loc[test["Municipality"] == municip]["Department"].iloc[0]
  counts = test[test["Municipality"] == municip].Pred_Fraud_type.value_counts()
  if (len(counts) > 2):
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Municipality"] == municip]["Turnout"].mean()
    avg_vote = test[test["Municipality"] == municip]["MAS.vote"].mean()
    avg_ensemble_municip_results.loc[len(avg_ensemble_municip_results)] = [municip, dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
avg_ensemble_municip_results = avg_ensemble_municip_results.sort_values(by = "Clean")
avg_ensemble_municip_results.reset_index(drop=True, inplace=True)

In [19]:
avg_ensemble_municip_results.to_csv("ensemble_avg_municip_results.csv", index=False)
avg_ensemble_municip_results

,Municipality,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Chacarilla,La Paz,33.33,66.67,33.33,33.33,90.33,0.81
1,Malla,La Paz,33.33,66.67,33.33,33.33,96.27,0.83
2,Pojo,Cochabamba,42.86,57.14,42.86,14.29,92.67,0.88
3,Sacaca,Potosí,45.45,54.55,30.30,24.24,93.73,0.85
4,Papel Pampa,La Paz,45.45,54.55,36.36,18.18,90.87,0.82
...,...,...,...,...,...,...,...,...
117,Tarija,Tarija,96.18,3.82,3.27,0.55,88.68,0.31
118,Santa Cruz de La Sierra,Santa Cruz,96.50,3.50,3.37,0.13,88.90,0.28
119,Nuestra Señora de La Paz,La Paz,96.67,3.33,2.95,0.39,91.00,0.40
120,Montero,Santa Cruz,96.69,3.31,2.21,1.10,88.64,0.39


#Ensemble (voting approach)

In [20]:
seed = 8
voting_model = VotingClassifier(estimators = [("RF", rf_model), 
                                              ("XGB", xgb_model), 
                                              ("KNN", knn_model), 
                                              ("LogR", logr_model), 
                                              ("GB", gb_model)], 
                                voting="soft")
voting_model.fit(X_train, y_train)

VotingClassifier(estimators=[('RF', RandomForestClassifier()),
                             ('XGB', XGBClassifier(objective='multi:softprob')),
                             ('KNN', KNeighborsClassifier()),
                             ('LogR', LogisticRegression(max_iter=1000)),
                             ('GB', GradientBoostingClassifier())],
                 voting='soft')

In [21]:
voting_pred = voting_model.predict(X_test)

In [22]:
test["Pred_Fraud_num"] = voting_pred.tolist()
test["Pred_Fraud_type"] = [ "VS" if type == -1 else "BBS" if type == 1 else "Clean" for type in test["Pred_Fraud_num"] ]
test["Department"] = testing_data["Department"]
test["Precinct"] = testing_data["Precinct"]
test["Municipality"] = testing_data["Municipality"]

In [23]:
test.to_csv("ensemble_voting_results.csv", index=False)

In [24]:
col_names = ["Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
voting_ensemble_results = pd.DataFrame(columns = col_names)
counts = test.Pred_Fraud_type.value_counts()
clean = counts[0] / (counts[0] + counts[1] + counts[2])
tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
bbs = counts[1] / (counts[0] + counts[1] + counts[2])
vs = counts[2] / (counts[0] + counts[1] + counts[2])
avg_turnout = test["Turnout"].mean()
avg_vote = test["MAS.vote"].mean()
voting_ensemble_results.loc[len(avg_ensemble_results)] = [round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]

In [25]:
voting_ensemble_results.to_csv("ensemble_voting_tot_results.csv", index=False)
voting_ensemble_results

,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
1,86.18,13.82,9.2,4.62,89.8,0.46


In [26]:
col_names = ["Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
voting_ensemble_dep_results = pd.DataFrame(columns = col_names)
for dep in test["Department"].unique():
    vars()[dep] = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    counts = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Department"] == dep]["Turnout"].mean()
    avg_vote = test[test["Department"] == dep]["MAS.vote"].mean()
    voting_ensemble_dep_results.loc[len(voting_ensemble_dep_results)] = [dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
voting_ensemble_dep_results = voting_ensemble_dep_results.sort_values(by = "Clean")
voting_ensemble_dep_results.reset_index(drop=True, inplace=True)

In [27]:
voting_ensemble_dep_results.to_csv("ensemble_voting_dep_results.csv", index=False)
voting_ensemble_dep_results

,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Cochabamba,77.41,22.59,11.32,11.28,90.28,0.57
1,La Paz,83.91,16.09,12.88,3.21,91.75,0.53
2,Oruro,86.68,13.32,11.18,2.13,91.09,0.47
3,Potosí,86.80,13.20,6.72,6.48,88.87,0.46
4,Santa Cruz,90.54,9.46,7.53,1.93,88.83,0.34
5,Chuquisaca,91.50,8.50,5.62,2.88,88.41,0.40
6,Tarija,94.36,5.64,2.85,2.79,87.26,0.39
7,Beni,94.78,5.22,2.89,2.33,86.17,0.34
8,Pando,95.42,4.58,3.05,1.53,84.31,0.43


In [28]:
col_names = ["Municipality", "Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
voting_ensemble_municip_results = pd.DataFrame(columns = col_names)
for municip in test["Municipality"].unique():
  dep = test.loc[test["Municipality"] == municip]["Department"].iloc[0]
  counts = test[test["Municipality"] == municip].Pred_Fraud_type.value_counts()
  if (len(counts) > 2):
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Municipality"] == municip]["Turnout"].mean()
    avg_vote = test[test["Municipality"] == municip]["MAS.vote"].mean()
    voting_ensemble_municip_results.loc[len(voting_ensemble_municip_results)] = [municip, dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
voting_ensemble_municip_results = voting_ensemble_municip_results.sort_values(by = "Clean")
voting_ensemble_municip_results.reset_index(drop=True, inplace=True)

In [29]:
voting_ensemble_municip_results.to_csv("ensemble_voting_municip_results.csv", index=False)
voting_ensemble_municip_results

,Municipality,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Chacarilla,La Paz,33.33,66.67,33.33,33.33,90.33,0.81
1,Caripuyo,Potosí,38.46,61.54,38.46,23.08,90.48,0.84
2,Pojo,Cochabamba,42.86,57.14,33.33,23.81,92.67,0.88
3,Colquechaca,Potosí,45.00,55.00,42.50,12.50,89.21,0.79
4,Papel Pampa,La Paz,45.45,54.55,36.36,18.18,90.87,0.82
...,...,...,...,...,...,...,...,...
127,Tupiza,Potosí,95.45,4.55,3.64,0.91,87.35,0.56
128,Tarija,Tarija,95.63,4.37,4.09,0.27,88.68,0.31
129,Bermejo,Tarija,95.97,4.03,3.23,0.81,84.90,0.45
130,San Lorenzo,Tarija,97.22,2.78,1.39,1.39,87.64,0.44
